FIRST STEP: getting data from balance sheets

In [1]:
import pandas as pd
import numpy as np
import os
import json

oscwd = os.getcwd()

# Load dataframe with BALANCE SHEET data (assets, liabilities and equity)
df_evoldatos = pd.read_csv(os.path.join(oscwd, "data/dbo_Men_EvolDatos.txt"),
                            dtype = {
                                    'Entidad': str, 'Periodo': str, 
                                    'Cuenta': str, 'Saldo': float
                                    }
                                    # parse_dates=['Periodo'],
                            )

# Load dataframe with INCOME STATEMENT data
df_evolresul = pd.read_csv(os.path.join(oscwd, "data/dbo_Men_EvolResul.txt"),
                            dtype = {
                                    'Entidad': str, 'Periodo': str, 
                                    'Cuenta': str, 'Saldo': float
                                    }
                                    # parse_dates=['Periodo'],
                            )

# Load dataframe with SPECIAL data needed for personal loans
df_evolintdocpers = pd.read_csv(os.path.join(oscwd, "data/dbo_Men_IntDocPers.txt"),
                            dtype = {
                                    'Codent': str, 'FEC_INF': str, 
                                    'PARTIDA': str, 'SumaDeIMPORTE': float
                                    }
                                    # parse_dates=['Periodo'],
                            )

# Load dataframe with average weighted value of loan segments
df_avg_loan_seg_weighted_value = pd.read_csv(os.path.join(oscwd, "data/df_loan_seg_avg_wv.csv"),
                            dtype = {
                                    'bank': str, 'per': str, 
                                    'acc': str, 'weighted_value': float
                                    }
                                    # parse_dates=['Periodo'],
                            )

# Renaming columns
new_columns = ["bank", "date", "acc", "total"]
df_evoldatos.columns = new_columns
df_evolresul.columns = new_columns
df_evolintdocpers.columns = new_columns
df_avg_loan_seg_weighted_value.columns = new_columns

# Yearly column for later aggregation
df_evoldatos["per"] = [d[0:4] for d in df_evoldatos["date"]]
df_evolresul["per"] = [d[0:4] for d in df_evolresul["date"]]
df_evolintdocpers["per"] = [d[0:4] for d in df_evolintdocpers["date"]]
df_avg_loan_seg_weighted_value["per"] = [d[0:4] for d in df_avg_loan_seg_weighted_value["date"]]

In [2]:
df_asset_total = df_evoldatos.loc[df_evoldatos.acc=="10000000000"]
df_asset_total = df_asset_total.groupby(["date", "acc","per"]).sum("total").reset_index()
df_asset_total = df_asset_total.groupby(["acc", "per"]).mean("total").reset_index()
df_asset_total["bank"] = "11111"
df_asset_total["acc"] = "10000000000_"+str(df_asset_total["bank"][0])
df_asset_total = df_asset_total.iloc[:,[3,0,1,2]]

We will be using operative banks (excluding those who have shut)

In [3]:
# Load dataframe with operative banks
active = pd.read_csv(os.path.join(oscwd, "data/vigentes.csv"),
                        dtype = {
                                'EENTIDAD': str
                        }
                        )
active.columns = ["bank"]

In [4]:
# Create list of operative banks that are actually in the balance sheet statement
banks = [a for a in active["bank"] if a in df_evoldatos["bank"].unique()]

SECOND STEP: loading balance sheet accounts for each kpi from a json file

In [5]:
with open("data/kpi.json", "r") as kpi_json:
    kpi_ind = json.load(kpi_json)

num_list = list()
num_sign = list()
den_list = list()
den_sign = list()

for k in kpi_ind:
    num_list.append(kpi_ind[k]["num"]["n_acc"])
    num_sign.append(kpi_ind[k]["num"]["sign"])
    den_list.append(kpi_ind[k]["den"]["n_acc"])
    den_sign.append(kpi_ind[k]["den"]["sign"])

THIRD STEP: filtering, aggregating and concatenating data.

We will remove non operative banks and also aggregate the data: grouping and averaging BALANCE SHEET accounts, while adding accounts for INCOME STATEMENT

In [6]:
# Filtering by operative banks
df_evoldatos = df_evoldatos[df_evoldatos["bank"].isin(banks)]
df_evolresul = df_evolresul[df_evolresul["bank"].isin(banks)]
df_evolintdocpers = df_evolintdocpers[df_evolintdocpers["bank"].isin(banks)]
df_avg_loan_seg_weighted_value = df_avg_loan_seg_weighted_value[df_avg_loan_seg_weighted_value["bank"].isin(banks)]

# Aggregating data
df_evoldatos_agg = df_evoldatos.groupby(['bank', 'acc', 'per']).mean('total')
df_evolresul_agg = df_evolresul.groupby(['bank', 'acc', 'per']).sum('total')
df_evolintdocpers_agg = df_evolintdocpers.groupby(['bank', 'acc', 'per']).mean('total')

In [7]:
# Reseting indexes
df_evoldatos_agg = df_evoldatos_agg.reset_index()
df_evolresul_agg = df_evolresul_agg.reset_index()
df_evolintdocpers_agg = df_evolintdocpers_agg.reset_index()

In [9]:
# df_test = pd.concat([df_evoldatos_agg[df_evoldatos_agg["Cuenta"]=="10000000000"], df_asset_total], axis=1, keys="Anual")
df_asset = df_evoldatos_agg[df_evoldatos_agg["acc"]=="10000000000"]
df_asset["total"] = [df_asset_total["total"].loc[df_asset_total.index[df_asset_total["per"]==p][0]] for p in df_asset["per"]]
df_asset["acc"] = "10000000000_11111"

C:\Users\MEC\AppData\Local\Temp\ipykernel_15784\1280478950.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asset["total"] = [df_asset_total["total"].loc[df_asset_total.index[df_asset_total["per"]==p][0]] for p in df_asset["per"]]
C:\Users\MEC\AppData\Local\Temp\ipykernel_15784\1280478950.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asset["acc"] = "10000000000_11111"


In [11]:
# Concatenating and filtering out any data prior to ....
df_evol = pd.concat([df_evoldatos_agg, df_evolresul_agg, df_evolintdocpers_agg, df_asset, df_avg_loan_seg_weighted_value])
df_evol = df_evol[df_evol["per"]=="2015"]

FOURTH STEP: calculating kpis

In [12]:
per_list = [p for p in df_evol["per"].unique()]
acc_lists = [num_list[i]+den_list[i] for i in range(len(num_list))]
kpi_list = [kpi for kpi in kpi_ind.keys()]

df_ind = pd.DataFrame([], columns=["ent", "per", "ind", "val"])

for i, acc_list in enumerate(acc_lists):
    for p in per_list:
        try:
            df_pivot = df_evol[(df_evol["per"]==p)&(df_evol["acc"].isin(acc_list))].pivot(index=["bank"], columns="acc", values="total").reset_index()
            # df_pivot = df_pivot.fillna(0)
            df_pivot["num"] = (df_pivot[df_pivot.columns & num_list[i]] * [num_sign[i][num_list[i].index(acc_list)] for acc_list in df_pivot[df_pivot.columns & num_list[i]].columns]).sum(axis=1)
            df_pivot["den"] = (df_pivot[df_pivot.columns & den_list[i]] * [den_sign[i][den_list[i].index(acc_list)] for acc_list in df_pivot[df_pivot.columns & den_list[i]].columns]).sum(axis=1)
            df_pivot[kpi_list[i]] = df_pivot["num"] / df_pivot["den"]
            df_pivot[kpi_list[i]] = df_pivot[kpi_list[i]].replace([np.inf, -np.inf, ""], 0)
            df_pivot[kpi_list[i]] = df_pivot[kpi_list[i]].fillna(0)

            df_to_ind = pd.DataFrame({"ent": df_pivot[df_pivot.columns[0]], "per": p, "ind": kpi_list[i], "val": df_pivot[kpi_list[i]]})

            df_ind = df_ind.append(df_to_ind)
        except Exception as e:
            print(e)
            pass

df_ind.to_csv("data/kpis.csv", sep=",", index=False)

C:\Users\MEC\AppData\Local\Temp\ipykernel_15784\3441073303.py:12: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  df_pivot["num"] = (df_pivot[df_pivot.columns & num_list[i]] * [num_sign[i][num_list[i].index(acc_list)] for acc_list in df_pivot[df_pivot.columns & num_list[i]].columns]).sum(axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_15784\3441073303.py:13: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  df_pivot["den"] = (df_pivot[df_pivot.columns & den_list[i]] * [den_sign[i][den_list[i].index(acc_list)] for acc_list in df_pivot[df_pivot.columns & den_list[i]].columns]).sum(axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_15784\3441073303.py:20: FutureWarning: The frame.append method is deprecated and will 